In [597]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import math
from sklearn.model_selection import train_test_split
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from dateutil.rrule import rrule, DAILY 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score 
import math 
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import FunctionTransformer 
from sklearn.compose import ColumnTransformer 
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import re
from sklearn.preprocessing import OrdinalEncoder
import nltk
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgbm

In [598]:
def remove_stop_words(data):
    # This function could remove all stop words like the, a, I
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(data) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        data = str.replace(data, i, ' ')
    return data

def remove_apostrophe(data):
    return str.replace(data, "'", "")

def remove_single_characters(data):
    new_text = ""
    words = word_tokenize(data)
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def stemming(data):
    return PorterStemmer().stem(data)

In [599]:
def preprocess(data):
    data = str(data)
    data = data.lower()
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    return data

In [600]:
# read section
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

C:\Users\xinrui zhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocessing part

In [624]:
# extra people
train_df.extra_people = train_df.extra_people.apply(lambda x: int(x[:-3].replace('$', '')))
test_df.extra_people = test_df.extra_people.apply(lambda x: int(x[:-3].replace('$', '')))
def trans_ep(ele):
    if ele == 0:
        return 0
    elif ele < 10:
        return 1
    elif ele < 20:
        return 2
    elif ele < 30:
        return 3
    elif ele < 50:
        return 4
    elif ele < 80:
        return 5
    elif ele < 150:
        return 6
    else:
        return 7
train_df.extra_people = train_df.extra_people.apply(trans_ep)
test_df.extra_people = test_df.extra_people.apply(trans_ep)

In [602]:
# summary , words are picked from tfidf
train_df.summary = train_df.summary.apply(preprocess)
test_df.summary = test_df.summary.apply(preprocess)
summary_lst_high = ['sonder', 'luxurious', '45th', 'kettl', 'typically', 'LUX', 'superbowl', 'floorplan', 'wyndham']
summary_lst_low = ['cat', 'shared', 'roommate', 'airline', 'bedly', 'turnkey']
def extract_sum_1(summary):
    words = word_tokenize(summary)
    res1 = 0
    for i in summary_lst_high:
        if i in words:
            res1 += 1
    return res1
def extract_sum_2(summary):
    words = word_tokenize(summary)
    res2 = 0
    for i in summary_lst_low:
        if i in words:
            res2 += 1
    return res2

train_df['summary_1'] = train_df.summary.apply(extract_sum_1)
test_df['summary_1'] = test_df.summary.apply(extract_sum_1)
train_df['summary_2'] = train_df.summary.apply(extract_sum_2)
test_df['summary_2'] = test_df.summary.apply(extract_sum_2)

In [603]:
# description, words are picked from tfidf
train_df.description = train_df.description.apply(preprocess)
test_df.description = test_df.description.apply(preprocess)
des_lst_high = ['superbowl', 'cinemax', 'hard', 'nearest', 'matter']
des_lst_low = ['laundromat', 'cat', 'laguardia', 'female', 'fan', ]
def extract_des_1(summary):
    words = word_tokenize(summary)
    res1 = 0
    for i in summary_lst_high:
        if i in words:
            res1 += 1
    return res1

def extract_des_2(summary):
    words = word_tokenize(summary)
    res2 = 0
    for i in summary_lst_low:
        if i in words:
            res2 += 1
    return res2

train_df['des_1'] = train_df.summary.apply(extract_des_1)
test_df['des_1'] = test_df.summary.apply(extract_des_1)
train_df['des_2'] = train_df.summary.apply(extract_des_2)
test_df['des_2'] = test_df.summary.apply(extract_des_2)

In [605]:
# transit, words are picked from tfidf
train_df.transit = train_df.transit.apply(preprocess)
test_df.transit = test_df.transit.apply(preprocess)trans_lst_high = ['wyndham', '•34', 'rockette', 'highline', 'sampling', 'headquarters', 'kicks', '•nyc',   ]
trans_lst_low = ['along', 'utica', 'road', 'transfer', 'myrtle', 'av', 'parkway', 'quick', ]
def extract_trans_1(summary):
    words = word_tokenize(summary)
    res1 = 0
    for i in trans_lst_high:
        if i in words:
            res1 += 1
    return res1

def extract_trans_2(summary):
    words = word_tokenize(summary)
    res2 = 0
    for i in trans_lst_low:
        if i in words:
            res2 += 1
    return res2

train_df['trans_1'] = train_df.summary.apply(extract_trans_1)
test_df['trans_1'] = test_df.summary.apply(extract_trans_1)
train_df['trans_2'] = train_df.summary.apply(extract_trans_2)
test_df['trans_2'] = test_df.summary.apply(extract_trans_2)

In [606]:
# zipcode, only using the first five
train_df.zipcode = train_df.zipcode.apply(lambda x: str(x)[:5])
test_df.zipcode = test_df.zipcode.apply(lambda x: str(x)[:5])

In [607]:
# self one hot the amenities, 
train_df.amenities = train_df.amenities.apply(preprocess).apply(lambda x: word_tokenize(x))
test_df.amenities = test_df.amenities.apply(preprocess).apply(lambda x: word_tokenize(x))
amenities_dct = {}
for lst in train_df.amenities:
    for item in lst:
        if item not in amenities_dct.keys():
            amenities_dct[item] = 1
        else:
            amenities_dct[item]+=1
values = np.array(list(amenities_dct.values()))
keys = np.array(list(amenities_dct.keys()))
ind = (values > 500) & (values < 34000)
display(len(keys[ind]))
related = keys[ind]

def one_hot(lst):
    temp = np.zeros(157)
    for i in range(len(related)):
        if related[i] in lst:
            temp[i] = 1
    return temp

amenities_oh = train_df.amenities.apply(one_hot)
train_df['amenities_oh'] = amenities_oh
test_df['amenities_oh'] = test_df.amenities.apply(one_hot)

In [610]:
# whether a real bed
def whether_realbed(bed):
    if bed == 'Real Bed':
        return 1
    else:
        return 0
train_df.bed_type = train_df.bed_type.apply(whether_realbed)
test_df.bed_type = test_df.bed_type.apply(whether_realbed)

In [721]:
# train, test, split
train_X, val_X, y_train, y_val = train_test_split(train_df.drop(columns=['price']), train_df.price, test_size = 0.5)
train_X.loc[:, 'Y'] = y_train
val_X.loc[:, 'Y'] = y_val

In [726]:
# This section is reserved for later. Since sklearn ordinal coding does not have handle unknown option, we need to manually
# change the unseen value to most frequent before we run.
val_X.zipcode = val_X.zipcode.apply(lambda x: '11211' if x in ['10119', '91766', '10162', '11621', '11461', '10281', '10106', '11363']  else x)

### Model

In [612]:
# prepare transform function for late pipeline
property_list = ['Apartment', 'House', 'Townhouse', 'Loft', 'Condominium', 'Serviced apartment', 'Guest suite', 'none']
def prop_help(ele):
    if ele in property_list:
        return ele
    else:
        return 'else'
    
def tran_proper(col):
    col = col.fillna('none')
    col.property_type = col.property_type.apply(prop_help)
    return col

nei_lst = list(train_df.neighbourhood_cleansed.value_counts().index[:10])
def transform_help(ele):
    if ele in nei_lst:
        return ele
    else:
        return 'else'
    
def transform_nei(col):
    col = col.fillna('none')
    col.neighbourhood_cleansed = col.neighbourhood_cleansed.apply(transform_help)
    return col

In [735]:
# preprocessing pipeline
num_feat = ['accommodates', 'beds', 'bathrooms', 'bedrooms', 'reviews_per_month', 'guests_included', 'minimum_nights', 'maximum_nights']
num_t = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values=np.nan, strategy='mean'))
])

str_feat = ['room_type', 'review_scores_location', 'review_scores_cleanliness']
str_t = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())
])

nei_feat_gr = ['neighbourhood_group_cleansed']
nei_t_gr = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown = 'ignore', sparse=False))
])

prop_feat = ['property_type']
prop_t = Pipeline(steps=[
    ('property', FunctionTransformer(tran_proper, validate=False)),
    ('ordinal', OrdinalEncoder())
])

sum_feat = ['summary_1', 'summary_2']
sum_t = Pipeline(steps=[
    ('property', FunctionTransformer(lambda x: x, validate=False))
])

des_feat = ['des_1', 'des_2']
des_t = Pipeline(steps=[
    ('property', FunctionTransformer(lambda x: x, validate=False))
])

nei_feat = ['neighbourhood_cleansed']
nei_t = Pipeline(steps=[
    ('nei', FunctionTransformer(transform_nei, validate=False)),
    ('ordinal', OrdinalEncoder())
])

zip_feat = ['zipcode']
zip_t = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())
])

extra_feat = ['extra_people']
extra_t = Pipeline(steps=[
    ('imp', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('property', FunctionTransformer(lambda x: x, validate=False))
    #('ordinal', OrdinalEncoder())
])

amenities_feat = ['amenities_oh']
amenities_t = Pipeline(steps=[
    ('property', FunctionTransformer(lambda x: x, validate=False))
])

bed_feat = ['bed_type']
bed_t = Pipeline(steps=[
    ('property', FunctionTransformer(lambda x: x, validate=False))
])

trans_feat = ['trans_1', 'trans_2']
trans_t = Pipeline(steps=[
    ('property', FunctionTransformer(lambda x: x, validate=False))
])

preproc = ColumnTransformer(transformers=[
    ('num', num_t, num_feat),
    ('str', str_t, str_feat),
    ('nei_gr', nei_t_gr, nei_feat_gr),
    ('prop', prop_t, prop_feat),
    ('sum', sum_t, sum_feat),
    ('nei', nei_t, nei_feat),
    ('des', des_t, des_feat),
    ('zip', zip_t, zip_feat),
    ('extra', extra_t, extra_feat),
    ('bed', bed_t, bed_feat),
    ('trans', trans_t, trans_feat),
    ('amenities', amenities_t, amenities_feat)
])

plf = Pipeline(steps=[('prep', preproc)]) #, ('regr', rf)]

In [740]:
# fit whole data, this is called when I wanna submit
plf.fit(train_df.drop(columns=['price']), train_df['price'])

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imp',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                                           

In [736]:
# fit the train data, called when I am modeling
plf.fit(train_X.drop(columns=['Y']), train_X['Y'])

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imp',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                                           

In [737]:
# same with the reserved section, this is used for handling the ordinal encoding
test_df.zipcode = test_df.zipcode.apply(lambda x: '11211' if x in ['10704', '11954', '10550', '10174', '1m', '11239', '10107', '11121'] else x)
#plf.transform(test_df)

In [741]:
# this will flatt the self coded onehot econding return array
def flatt(a):
    new = []
    for sa in a:
        new.append(np.concatenate((sa[:-1], sa[-1]), axis=None))
    return np.array(new)

In [753]:
# light gbm model parameters
params = {
        'boosting_type':'dart',
        'objective': 'regression',
        'num_leaves': 42,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'bagging_freq': 20,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_data_in_leaf': 5,
        'scale_pos_weight':1,
        'num_rounds':1500,
        'lambda_l2': 0.05,
        'colsample_bytree': 0.6,
        'xgboost_dart_mode': True,
    }
# below two will called when I wanna submit
train_set = lgbm.Dataset(flatt(plf.transform(train_df)), train_df['price'])
valid_set = lgbm.Dataset(flatt(plf.transform(train_df)), train_df['price'])

# below two will called when modeling
#train_set = lgbm.Dataset(flatt(plf.transform(train_X)), train_X['Y'])
#valid_set = lgbm.Dataset(flatt(plf.transform(val_X)), val_X["Y"])

model = lgbm.train(params, train_set = train_set, 
                   verbose_eval=20, valid_sets=valid_set)

C:\Users\xinrui zhan\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	valid_0's rmse: 97.6807
[40]	valid_0's rmse: 86.9255
[60]	valid_0's rmse: 82.7402
[80]	valid_0's rmse: 80.0205
[100]	valid_0's rmse: 78.1961
[120]	valid_0's rmse: 76.5421
[140]	valid_0's rmse: 74.9907
[160]	valid_0's rmse: 74.3162
[180]	valid_0's rmse: 73.4645
[200]	valid_0's rmse: 71.9521
[220]	valid_0's rmse: 70.5647
[240]	valid_0's rmse: 69.6749
[260]	valid_0's rmse: 68.9227
[280]	valid_0's rmse: 67.9565
[300]	valid_0's rmse: 67.1433
[320]	valid_0's rmse: 66.3346
[340]	valid_0's rmse: 65.7866
[360]	valid_0's rmse: 65.1429
[380]	valid_0's rmse: 64.4683
[400]	valid_0's rmse: 63.9526
[420]	valid_0's rmse: 63.6448
[440]	valid_0's rmse: 63.0612
[460]	valid_0's rmse: 62.7018
[480]	valid_0's rmse: 62.2286
[500]	valid_0's rmse: 61.7055
[520]	valid_0's rmse: 61.2082
[540]	valid_0's rmse: 60.7847
[560]	valid_0's rmse: 60.4525
[580]	valid_0's rmse: 60.1931
[600]	valid_0's rmse: 59.7693
[620]	valid_0's rmse: 59.0865
[640]	valid_0's rmse: 58.558
[660]	valid_0's rmse: 58.1447
[680]	valid_0's

### Save

In [756]:
# for ordinal encoding
test_df.loc[test_df.review_scores_location == 3, 'review_scores_location'] = 2

prediction = model.predict(flatt(plf.transform(test_df)), num_iteration = model.best_iteration)
test_df['Predicted'] =  pd.Series(prediction)

test_df.loc[:, ['id', 'Predicted']].to_csv('boosting.csv', index=False)
boot = pd.read_csv('boosting.csv')

In [668]:
boot[boot.id == 25423395]['Predicted']

2521    31.796377
Name: Predicted, dtype: float64